# Dataset Creation

In [1]:
!pip install gTTS


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
from gtts import gTTS
import os
from tqdm import tqdm
from pydub import AudioSegment

In [4]:
data = pd.read_csv("D:\\IIIT Delhi\\7th Semester\\Courses\\BTP\\transcript.csv").head(1500)
data

,Unnamed: 0,transcript
0,0,I have to say that I'm very encouraged by the ...
1,1,Our Fixed Income franchise continued to benefi...
2,2,"As a sign of our commitment to Mexico, a marke..."
3,3,The amount of outstanding is 2% lower from the...
4,4,the uncertainties that color the current econo...
...,...,...
1495,1495,by blocking more threats and indeed more advan...
1496,1496,We took a very different approach by applying ...
1497,1497,And that is the ability for the firewall and e...
1498,1498,"Now in addition to all this, the capability in..."


In [5]:
output_folder = "D:\\IIIT Delhi\\7th Semester\\Courses\\BTP\\Dataset\\data\\robot+legit\\"

In [ ]:
for idx, row in tqdm(data.iterrows(), total=len(data), desc="Processing Transcripts"):
    # Extract the transcript from the dataframe (replace 'transcript_column' with the actual column name)
    transcript = row['transcript']  # Replace with the actual column name in your DataFrame
    
    # Create the TTS object
    tts = gTTS(text=transcript, lang='en')  # You can change the language as needed

    # Save the audio as an MP3 file first
    mp3_file_path = os.path.join(output_folder, f"transcript_{idx+1}.mp3")
    tts.save(mp3_file_path)

    # Convert the MP3 file to 16kHz WAV using pydub
    audio = AudioSegment.from_mp3(mp3_file_path)
    audio = audio.set_frame_rate(16000)  # Set the sample rate to 16kHz

    # Export the audio as a WAV file
    wav_file_path = os.path.join(output_folder, f"transcript_{idx+1}.wav")
    audio.export(wav_file_path, format="wav")

    # Optionally, remove the temporary MP3 file
    os.remove(mp3_file_path)

In [24]:
tosave = data.head(1385)

In [27]:
tosave.drop('Unnamed: 0', axis=1, inplace=True)

C:\Users\abhij\AppData\Local\Temp\ipykernel_18408\3029695330.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tosave.drop('Unnamed: 0', axis=1, inplace=True)


In [30]:
transcript_ids = ['transcript_' + str(i + 1) for i in range(1385)]

In [31]:
tosave.insert(0, 'transcript_id', transcript_ids)


In [33]:
tosave

,transcript_id,transcript
0,transcript_1,I have to say that I'm very encouraged by the ...
1,transcript_2,Our Fixed Income franchise continued to benefi...
2,transcript_3,"As a sign of our commitment to Mexico, a marke..."
3,transcript_4,The amount of outstanding is 2% lower from the...
4,transcript_5,the uncertainties that color the current econo...
...,...,...
1380,transcript_1381,"If in the meantime, you have any questions, as..."
1381,transcript_1382,we have once again a record crowd joining our ...
1382,transcript_1383,"the input cost as well. I have to say, the cha..."
1383,transcript_1384,remind everybody where we are. We are in the p...


In [34]:
tosave.to_csv("D:\\IIIT Delhi\\7th Semester\\Courses\\BTP\\Dataset\\data\\try.csv")

# Lowpass Filter and Adding Noise

In [1]:
import os
import numpy as np
import librosa
import scipy.signal as signal
import soundfile as sf
from tqdm import tqdm

In [2]:
# Function to apply a low-pass filter
def lowpass_filter(audio, sample_rate, cutoff=3500):
    nyquist = 0.5 * sample_rate
    normal_cutoff = cutoff / nyquist
    b, a = signal.butter(4, normal_cutoff, btype='low')
    filtered_audio = signal.filtfilt(b, a, audio)
    return filtered_audio

# Function to add white noise
def add_white_noise(audio, noise_level=0.01):
    noise = np.random.normal(0, noise_level, len(audio))
    noisy_audio = audio + noise
    return noisy_audio

In [3]:
input_dir = "D:\\IIIT Delhi\\7th Semester\\Courses\\BTP\\Dataset\\data\\robot+legit\\"
output_dir = "D:\\IIIT Delhi\\7th Semester\\Courses\\BTP\\Dataset\\data\\robot+legit\\processed\\"

In [4]:
os.makedirs(output_dir, exist_ok=True)

# Get list of all .wav files in the directory
audio_files = [f for f in os.listdir(input_dir) if f.endswith('.wav')]

In [5]:
audio_files[0]

'transcript_1.wav'

In [6]:
for file_name in tqdm(audio_files, desc="Processing audio files"):
    # Load the audio file
    file_path = os.path.join(input_dir, file_name)
    audio, sample_rate = librosa.load(file_path, sr=None)

    # Apply low-pass filter
    filtered_audio = lowpass_filter(audio, sample_rate, cutoff=3500)

    # Add white noise
    noisy_audio = add_white_noise(filtered_audio, noise_level=0.01)

    # Save the processed audio
    output_path = os.path.join(output_dir, file_name)
    sf.write(output_path, noisy_audio, sample_rate)

Processing audio files: 100%|██████████| 1384/1384 [01:38<00:00, 14.04it/s]
